In [7]:
# Movie Recommendation System

In [19]:
# 1. load Data

# 2. Preprocessing:
# understand your data
# Create a list of important columns for recommendation colums,
# what features are important?
# name,	rating,	genre, votes,	director,	writer,	star,	gross ; these are imporatnt
# clean data (missing value, null values, errors, wrong information, etc )
# create a function

# 3. Model Building

In [6]:
# load Libraries
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# load Data
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies.csv


In [5]:
# convert to dataframe
df = pd.read_csv("movies.csv")
# Hava quick look at the data
df.head(6)

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0
5,Friday the 13th,R,Horror,1980,"May 9, 1980 (United States)",6.4,123000.0,Sean S. Cunningham,Victor Miller,Betsy Palmer,United States,550000.0,39754601.0,Paramount Pictures,95.0


In [8]:
# get a count of number of rows and colums in data
df.shape


(7668, 15)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7668 entries, 0 to 7667
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      7668 non-null   object 
 1   rating    7591 non-null   object 
 2   genre     7668 non-null   object 
 3   year      7668 non-null   int64  
 4   released  7666 non-null   object 
 5   score     7665 non-null   float64
 6   votes     7665 non-null   float64
 7   director  7668 non-null   object 
 8   writer    7665 non-null   object 
 9   star      7667 non-null   object 
 10  country   7665 non-null   object 
 11  budget    5497 non-null   float64
 12  gross     7479 non-null   float64
 13  company   7651 non-null   object 
 14  runtime   7664 non-null   float64
dtypes: float64(5), int64(1), object(9)
memory usage: 898.7+ KB


In [11]:
# understand your data
df.describe()

,year,score,votes,budget,gross,runtime
count,7668.000000,7665.000000,7.665000e+03,5.497000e+03,7.479000e+03,7664.000000
mean,2000.405451,6.390411,8.810850e+04,3.558988e+07,7.850054e+07,107.261613
std,11.153508,0.968842,1.633238e+05,4.145730e+07,1.657251e+08,18.581247
min,1980.000000,1.900000,7.000000e+00,3.000000e+03,3.090000e+02,55.000000
25%,1991.000000,5.800000,9.100000e+03,1.000000e+07,4.532056e+06,95.000000
50%,2000.000000,6.500000,3.300000e+04,2.050000e+07,2.020576e+07,104.000000
75%,2010.000000,7.100000,9.300000e+04,4.500000e+07,7.601669e+07,116.000000
max,2020.000000,9.300000,2.400000e+06,3.560000e+08,2.847246e+09,366.000000


In [12]:
# create a list of important colums for the recommendation engine
columns = ["name", "rating", "genre", "votes", "director", "writer", "star", "gross"]
# show colums
df[columns].head(6)

,name,rating,genre,votes,director,writer,star,gross
0,The Shining,R,Drama,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,46998772.0
1,The Blue Lagoon,R,Adventure,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,58853106.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,538375067.0
3,Airplane!,PG,Comedy,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,83453539.0
4,Caddyshack,R,Comedy,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,39846344.0
5,Friday the 13th,R,Horror,123000.0,Sean S. Cunningham,Victor Miller,Betsy Palmer,39754601.0


In [14]:
# clean data, check for missing values,
df[columns].isnull().values.any()

True

In [15]:
# Check for missing values
if df[columns].isnull().values.any():
    print("There are missing values in the dataset.")
    # Handle missing values
    # For example, you can replace missing values with a specific value or use imputation techniques

    # Replace missing values with a specific value (e.g., 0)
    df.fillna(0, inplace=True)
    print("Missing values have been filled with 0.")

    # Alternatively, you can drop rows with missing values
    # df.dropna(inplace=True)
    # print("Rows with missing values have been dropped.")
else:
    print("There are no missing values in the dataset.")


There are missing values in the dataset.
Missing values have been filled with 0.


In [16]:
# create a function to combine value of important columns into a string
def get_important_features(data):
    important_features = []
    for i in range(0, data.shape[0]):
        important_features.append(f"{data['name'][i]} {data['rating'][i]} {data['votes'][i]} {data['director'][i]} {data['writer'][i]} {data['star'][i]} {data['gross'][i]}")
    return important_features

In [17]:
# create a column to hold the combined strings
# Create a new column to hold the combined strings
df["important_features"] = get_important_features(df)

In [18]:
# Always check the work being done to DataFrame, in this case the new column
df.head(6)

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,important_features
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,The Shining R 927000.0 Stanley Kubrick Stephen...
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,The Blue Lagoon R 65000.0 Randal Kleiser Henry...
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,Star Wars: Episode V - The Empire Strikes Back...
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0,Airplane! PG 221000.0 Jim Abrahams Jim Abraham...
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0,Caddyshack R 108000.0 Harold Ramis Brian Doyle...
5,Friday the 13th,R,Horror,1980,"May 9, 1980 (United States)",6.4,123000.0,Sean S. Cunningham,Victor Miller,Betsy Palmer,United States,550000.0,39754601.0,Paramount Pictures,95.0,Friday the 13th R 123000.0 Sean S. Cunningham ...


In [20]:
# convert text into a matrix of token counts
cm = CountVectorizer().fit_transform(df["important_features"])

In [22]:
# Get a cosine similarity matrix from the count matrix
cs = cosine_similarity(cm)
# print the cosine similarity matrix
# 1 being 100% similarity to a movie
# Be careful the 1 is the same movies hence 100% similarity
print(cs)

[[1.         0.0877058  0.07905694 ... 0.         0.         0.        ]
 [0.0877058  1.         0.06933752 ... 0.         0.         0.        ]
 [0.07905694 0.06933752 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [23]:
# Get the shape of the cosine similarity matrix
# important that they are equal
cs.shape

(7668, 7668)

In [26]:
# Get the title of the movie that the user likes
title = "Star Wars: Episode V - The Empire Strikes Back"

# Our dataset doesnt have a unique movie id, so lets create a new column 'movie_id' with unique IDs
df['movie_id'] = range(1, len(df) + 1)

df.head(3)

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,important_features,movie_id
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0,The Shining R 927000.0 Stanley Kubrick Stephen...,1
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0,The Blue Lagoon R 65000.0 Randal Kleiser Henry...,2
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0,Star Wars: Episode V - The Empire Strikes Back...,3


In [27]:
# Get the title of the movie that the user likes
title = "Star Wars: Episode V - The Empire Strikes Back"

# Find the movie's ID
movie_id = df[df.name == title]["movie_id"].values[0]

In [28]:
# create a list enumerations for the similarity score
# Create a list of tuples containing movie IDs and their similarity scores
# Assume cs is a NumPy array containing similarity scores
# and the movie IDs are stored in the first column of the array

# Get the column index for the movie ID
column_index_for_movie_id = 0  # Assuming the movie ID is in the first column

# Create a list of tuples containing movie IDs and their similarity scores
scores = list(enumerate(cs[:, column_index_for_movie_id]))

# Display the first few elements of the list
print(scores[:5])  # Print only the first 5 elements for demonstration


[(0, 0.9999999999999999), (1, 0.08770580193070293), (2, 0.07905694150420949), (3, 0.0), (4, 0.0)]


In [29]:
# Sort the list
sorted_scores = sorted(scores, key= lambda x:x[1], reverse=True)
sorted_scores = sorted_scores[1:] # we dont want the first movie as that the same movie
print(sorted_scores[:20])

[(1690, 0.40824829046386296), (878, 0.33806170189140666), (1045, 0.3265986323710904), (3446, 0.3265986323710904), (280, 0.30000000000000004), (3921, 0.30000000000000004), (2087, 0.2860387767736777), (2370, 0.2860387767736777), (2443, 0.2860387767736777), (3184, 0.2860387767736777), (3449, 0.2860387767736777), (3571, 0.2860387767736777), (4245, 0.27524094128159016), (499, 0.27386127875258315), (1061, 0.27386127875258315), (1063, 0.27386127875258315), (134, 0.2631174057921088), (4085, 0.2631174057921088), (5111, 0.2631174057921088), (7625, 0.2631174057921088)]


In [30]:
# Get the title of the movie that the user likes
title = "Star Wars: Episode V - The Empire Strikes Back"

# Find the movie's ID
user_movie_id = df[df.name == title]["movie_id"].values[0]

# Calculate the cosine similarity scores between the user's selected movie and all other movies
user_movie_index = user_movie_id - 1  # Adjust movie ID to index
similarities = cs[user_movie_index]

# Sort the similarity scores in descending order
sorted_indices = np.argsort(similarities)[::-1]

# Display details of the top 5 recommended movies
print("Top 5 Recommended Movies similar to", title, "are:")
for idx in sorted_indices[1:6]:  # Exclude the user's selected movie itself
    movie_details = df.iloc[idx]
    print("Name:", movie_details['name'])
    print("Rating:", movie_details['rating'])
    print("Genre:", movie_details['genre'])
    print("Votes:", movie_details['votes'])
    print("Director:", movie_details['director'])
    print("Writer:", movie_details['writer'])
    print("Star:", movie_details['star'])
    print("Gross:", movie_details['gross'])
    print("-----------------------------")



Top 5 Recommended Movies similar to Star Wars: Episode V - The Empire Strikes Back are:
Name: Star Wars: Episode VI - Return of the Jedi
Rating: PG
Genre: Action
Votes: 973000.0
Director: Richard Marquand
Writer: Lawrence Kasdan
Star: Mark Hamill
Gross: 475106177.0
-----------------------------
Name: Star Wars: Episode VII - The Force Awakens
Rating: PG-13
Genre: Action
Votes: 876000.0
Director: J.J. Abrams
Writer: Lawrence Kasdan
Star: Daisy Ridley
Gross: 2069521700.0
-----------------------------
Name: Star Wars: The Clone Wars
Rating: PG
Genre: Animation
Votes: 61000.0
Director: Dave Filoni
Writer: Henry Gilroy
Star: Matt Lanter
Gross: 68282844.0
-----------------------------
Name: Star Wars: Episode IX - The Rise of Skywalker
Rating: PG-13
Genre: Action
Votes: 394000.0
Director: J.J. Abrams
Writer: Chris Terrio
Star: Daisy Ridley
Gross: 1078232589.0
-----------------------------
Name: Star Wars: Episode I - The Phantom Menace
Rating: PG
Genre: Action
Votes: 748000.0
Director: Georg

In [31]:
# Offering a suprise recommendation
# Identify the user's general preferences based on their liked movie.
# Find movies that are dissimilar to the user's preferred movie but are similar to the user's general preferences.
# Recommend these unexpected movies to the user.
# Let's implement this approach:


In [32]:
# Get the title of the movie that the user likes
user_liked_movie = "Star Wars: Episode V - The Empire Strikes Back"

# Find the movie's ID
user_liked_movie_id = df[df.name == user_liked_movie]["movie_id"].values[0]

# Calculate the cosine similarity scores between the user's liked movie and all other movies
user_movie_index = user_liked_movie_id - 1  # Adjust movie ID to index
similarities = cs[user_movie_index]

# Sort the similarity scores in ascending order to find dissimilar movies
sorted_indices = np.argsort(similarities)

# Recommend the top 5 dissimilar movies that are similar to the user's general preferences
recommended_movies = []
count = 0
idx = 0
while count < 5:
    movie_idx = sorted_indices[idx]
    if movie_idx != user_movie_index:  # Exclude the user's liked movie itself
        movie_details = df.iloc[movie_idx]
        recommended_movies.append((movie_details['name'], movie_details['rating'], movie_details['genre'], movie_details['votes'], movie_details['director'], movie_details['writer'], movie_details['star'], movie_details['gross']))
        count += 1
    idx += 1

# Display the recommended unexpected movies
print("Top 5 Unexpected Movies you might like:")
for movie in recommended_movies:
    print("Name:", movie[0])
    print("Rating:", movie[1])
    print("Genre:", movie[2])
    print("Votes:", movie[3])
    print("Director:", movie[4])
    print("Writer:", movie[5])
    print("Star:", movie[6])
    print("Gross:", movie[7])
    print("-----------------------------")


Top 5 Unexpected Movies you might like:
Name: Tee em el
Rating: 0
Genre: Horror
Votes: 7.0
Director: Pereko Mosia
Writer: Pereko Mosia
Star: Siyabonga Mabaso
Gross: 0.0
-----------------------------
Name: Babe
Rating: G
Genre: Comedy
Votes: 119000.0
Director: Chris Noonan
Writer: Dick King-Smith
Star: James Cromwell
Gross: 254134910.0
-----------------------------
Name: Dead Man Walking
Rating: R
Genre: Crime
Votes: 90000.0
Director: Tim Robbins
Writer: Helen Prejean
Star: Susan Sarandon
Gross: 39363635.0
-----------------------------
Name: Black Death
Rating: R
Genre: Action
Votes: 45000.0
Director: Christopher Smith
Writer: Dario Poloni
Star: Eddie Redmayne
Gross: 382879.0
-----------------------------
Name: Johnny Mnemonic
Rating: R
Genre: Action
Votes: 68000.0
Director: Robert Longo
Writer: William Gibson
Star: Keanu Reeves
Gross: 19075720.0
-----------------------------
